In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

file_path = 'data.csv'

data = pd.read_csv(file_path)

# data["sex"].replace(['male', 'female'], [0, 1], inplace=True)
# data["smoker"].replace(['no', 'yes'], [0, 1],  inplace=True)
# data_model = data.drop("region", axis=1)
# print(data_model)

# region_dummies = pd.get_dummies(data['region'], prefix='is', prefix_sep='_')


# # Concaténer les colonnes binaires avec le DataFrame original
# data = pd.concat([data, region_dummies], axis=1)
# data = data.drop(['region'], axis=1)

# Vérification des informations manquantes et des doublons
missing_data = data.isnull().sum()
duplicates = data.duplicated().sum()
data = data.drop_duplicates()


# Afficher le DataFrame avec les nouvelles colonnes binaires
print(data.head())


   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520


In [49]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Lasso,
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


data.dropna(axis=0, inplace=True)


X = data.drop('charges', axis=1)
y = data['charges']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 80% pour train et 20% de test

print("Train set X", X_train.shape)
print("Train set Y", y_train.shape)
print("Test set X", X_test.shape)
print("Test set Y", y_test.shape)

Train set X (1069, 6)
Train set Y (1069,)
Test set X (268, 6)
Test set Y (268,)


In [50]:

# Identifier les colonnes catégories et numériques
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns


# Créer le pipeline pour les features numériques
numerical_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

# Créer le pipeline pour les features catégoriales
categorial_pipeline = Pipeline([
    ('encoder', OneHotEncoder(sparse=False))
])

# Combine les pipelines en utilisant ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical', numerical_pipeline, numerical_cols),
        ('categorial', categorial_pipeline, categorical_cols)
    ])




# Créer le pipeline final en ajoutant le model

LR_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regression', LinearRegression())
])

Lasso_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('Lasso', Lasso())
])


# On entraine les donnnées
LR_pipeline.fit(X_train, y_train)
Lasso_pipeline.fit(X_train, y_train)





/home/antoine/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/antoine/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['age', 'bmi', 'children'], dtype='object')),
                                                 ('categorial',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(sparse=False))]),
                                                  Index(['sex', 'smoker', 'region'], dtype='object'))])),
                ('Lasso', Lasso())])

In [51]:
# On voulait check les outputs du transformer
preprocessor.set_output(transform='pandas').fit_transform(X_train)

/home/antoine/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,numerical__age,numerical__bmi,numerical__children,categorial__sex_female,categorial__sex_male,categorial__smoker_no,categorial__smoker_yes,categorial__region_northeast,categorial__region_northwest,categorial__region_southeast,categorial__region_southwest
1114,-1.157680,-0.996928,-0.907908,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
968,-1.300619,-0.792762,0.766904,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
599,0.914926,1.154664,0.766904,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
170,1.701087,1.806837,-0.907908,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
275,0.557580,-0.651417,0.766904,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1096,0.843457,0.730628,0.766904,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1131,-0.871804,2.539188,0.766904,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
1295,-1.372088,-1.411872,-0.070502,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
861,-0.085643,-0.419974,1.604310,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [52]:
# On predicte
y_pred_LR = LR_pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred_LR)
r2 = r2_score(y_test, y_pred_LR)
rmse = np.sqrt(mse)
print(f" score du LR modèle : {LR_pipeline.score(X_test, y_test)}")
print(f"mse : {mse}")
print(f"r2 : {r2}")
print(f"rmse : {rmse}")

 score du LR modèle : 0.8069287081198014
mse : 35478020.67523556
r2 : 0.8069287081198014
rmse : 5956.3428943635845


/home/antoine/.local/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/home/antoine/.local/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [53]:
# On predicte
y_pred_Lasso = Lasso_pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred_Lasso)
r2 = r2_score(y_test, y_pred_Lasso)
rmse = np.sqrt(mse)
print(f" score du Lasso modèle : {Lasso_pipeline.score(X_test, y_test)}")
print(f"mse : {mse}")
print(f"r2 : {r2}")
print(f"rmse : {rmse}")

 score du Lasso modèle : 0.8068460476468182
mse : 35493210.038402736
r2 : 0.8068460476468182
rmse : 5957.617815738329


/home/antoine/.local/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(
/home/antoine/.local/lib/python3.10/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but Lasso was fitted without feature names
  warnings.warn(
